[Norvig's setup code](https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb)

In [1]:
#  Python 3.x
import re
import numpy as np
import math
import urllib.request

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache
from itertools   import permutations, combinations, chain, cycle, product, islice
from heapq       import heappop, heappush

def Input(day):
    "Open this day's input file."
    filename = './input{}.txt'.format(day)
    return open(filename)

def transpose(matrix): return zip(*matrix)

def first(iterable): return next(iter(iterable))

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

cat = ''.join

Ø   = frozenset() # Empty set
inf = float('inf')
BIG = 10 ** 999

def grep(pattern, lines):
    "Print lines that match pattern."
    for line in lines:
        if re.search(pattern, line):
            print(line)

def groupby(iterable, key=lambda it: it):
    "Return a dic whose keys are key(it) and whose values are all the elements of iterable with that key."
    dic = defaultdict(list)
    for it in iterable:
        dic[key(it)].append(it)
    return dic

def powerset(iterable):
    "Yield all subsets of items."
    items = list(iterable)
    for r in range(len(items)+1):
        for c in combinations(items, r):
            yield c

# 2-D points implemented using (x, y) tuples
def X(point): return point[0]
def Y(point): return point[1]

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def cityblock_distance(p, q=(0, 0)): 
    "City block distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

def euclidean_distance(p, q=(0, 0)): 
    "Euclidean (hypotenuse) distance between two points."
    return math.hypot(X(p) - X(q), Y(p) - Y(q))

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def astar_search(start, h_func, moves_func):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(previous, s)
        for s2 in moves_func(s):
            new_cost = path_cost[s] + 1
            if s2 not in path_cost or new_cost < path_cost[s2]:
                heappush(frontier, (new_cost + h_func(s2), s2))
                path_cost[s2] = new_cost
                previous[s2] = s
    return dict(fail=True, front=len(frontier), prev=len(previous))
                
def Path(previous, s): 
    "Return a list of states that lead to state s, according to the previous dict."
    return ([] if (s is None) else Path(previous, previous[s]) + [s])

# [Day 1](https://adventofcode.com/2018/day/1)

For example, if the device displays frequency changes of +1, -2, +3, +1, then starting from a frequency of zero, the following changes would occur:

Current frequency  0, change of +1; resulting frequency  1.
Current frequency  1, change of -2; resulting frequency -1.
Current frequency -1, change of +3; resulting frequency  2.
Current frequency  2, change of +1; resulting frequency  3.
In this example, the resulting frequency is 3.

Here are other example situations:

+1, +1, +1 results in  3
+1, +1, -2 results in  0
-1, -2, -3 results in -6
Starting with a frequency of zero, what is the resulting frequency after all of the changes in frequency have been applied?


In [15]:
def day1_pt1(in_):
  return np.sum(list(map(int, in_)))

day1_pt1(Input(1))

590

## Part 2
Here are other examples:

* +1, -1 first reaches 0 twice.
* +3, +3, +4, -2, -4 first reaches 10 twice.
* -6, +3, +8, +5, -6 first reaches 5 twice.
* +7, +7, -2, -7, -4 first reaches 14 twice.

What is the first frequency your device reaches twice?

In [35]:
def day1_pt2(in_):
  xor_ = set()
  sum_ = 0
  for i in itertools.cycle(map(int, in_)):
    xor_.add(sum_)
    sum_ = sum_ + i
    if sum_ in xor_:
      return sum_

day1_pt2(list(Input(1)))

83445

# [Day 2](https://adventofcode.com/2018/day/2)

For example, if you see the following box IDs:

* abcdef contains no letters that appear exactly two or three times.
* bababc contains two a and three b, so it counts for both.
* abbcde contains two b, but no letter appears exactly three times.
* abcccd contains three c, but no letter appears exactly two times.
* aabcdd contains two a and two d, but it only counts once.
* abcdee contains two e.
* ababab contains three a and three b, but it only counts once.

Of these box IDs, four of them contain a letter which appears exactly twice, and three of them contain a letter which appears exactly three times. Multiplying these together produces a **checksum of 4 * 3 = 12.**

What is the checksum for your list of box IDs?

In [8]:
def day2_pt1(in_):
  def count23(s):
    counts = {v: k for k, v in Counter(s).items() if v in (2, 3)}
    return (
      counts.get(2, None) is not None, 
      counts.get(3, None) is not None)
  
  twos, threes = 0, 0
  for c2, c3 in map(count23, in_):
    twos += c2
    threes += c3
  return twos * threes

day2_pt1(Input(2))

6000

In [11]:
def diff_letters(s1, s2):
  diff = []
  for i, (c1, c2) in enumerate(zip(s1, s2)):
    if c1 != c2:
      diff.append(i)
    if len(diff) > 1:
      break
  return diff
  
def day2_pt2(in_):
  for i in range(len(in_)):
    for j in range(i + 1, len(in_)):
      diffs = diff_letters(in_[i], in_[j])
      if len(diffs) == 1:
        diff = diffs[0]
        return in_[i][:diff] + in_[i][diff+1:]

day2_pt2(list(Input(2)))        

'pbykrmjmizwhxlqnasfgtycdv\n'